<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/220118_sm_know_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('/content/gdrive/MyDrive/know/train/*.csv'))]
know_test = [pd.read_csv(path) for path in sorted(glob('/content/gdrive/MyDrive/know/test/*.csv'))]
train_know = pd.read_csv('/content/gdrive/MyDrive/know/df_1')
test_know = pd.read_csv('/content/gdrive/MyDrive/know/df_test')

In [ ]:
train1 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2017.csv', encoding='utf-8')
train2 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2018.csv', encoding='utf-8')
train3 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2019.csv', encoding='utf-8')
train4 = pd.read_csv('/content/gdrive/MyDrive/know/train/KNOW_2020.csv', encoding='utf-8')

train3 = train3[train3['idx'] != 19871]

test1 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2017_test.csv', encoding='utf-8')
test2 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2018_test.csv', encoding='utf-8')
test3 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2019_test.csv', encoding='utf-8')
test4 = pd.read_csv('/content/gdrive/MyDrive/know/test/KNOW_2020_test.csv', encoding='utf-8')

submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv', encoding='utf-8')

In [ ]:
df = pd.concat([train1[['idx', 'knowcode']], train2[['idx', 'knowcode']], train3[['idx', 'knowcode']], train4[['idx', 'knowcode']]], axis=0)
df = df.reset_index(drop=True)
df

,idx,knowcode
0,0,825101
1,1,140204
2,2,140204
3,3,140601
4,4,140204
...,...,...
35229,35244,121105
35230,35245,26102
35231,35246,31301
35232,35247,154105


In [ ]:
my_dict = {}
for i in range(len(df['knowcode'].unique())):
  my_dict[df['knowcode'].unique()[i]] = i

In [ ]:
your_dict = {v:k for k,v in my_dict.items()}

In [ ]:
def label_to_target(text:str):
  return your_dict[text]

train_know['label_answer'] = train_know['label_answer'].map(label_to_target)
test_know['label_answer'] = test_know['label_answer'].map(label_to_target)

In [ ]:
test_know

,idx,text,label_answer
0,0,다른것으로 대체할 수 없는 업무 없다 컴퓨터 없다 없다 없다 비서학,29401
1,1,"제조업이 줄어들 것으로 생각되기 때문 없다 제품검사시스템,PC,엑셀 없다 없다 없다...",28402
2,2,"1인 미디어 증가 없다 오디션(편집프로그램), 나홀로(방송진행장비) 없다 광선, 홍...",416302
3,3,선호직업 아님 없다 컴퓨터 없다 없다 없다 화학,155306
4,4,잡지를 위한 정보나 뉴스는 고갈되지 않기 때문에 꾸준히 유지될 것이다 편집기자 뉴스...,412003
...,...,...,...
35226,35244,없다 전기는 일상생활에 꼭 필요한 것이라서 관련 인력 일자리는 변화가 없을 것이므로...,153106
35227,35245,온라인 설문 도입으로 테블릿을 통해 온라인으로 설문을 받는 방법과 에디팅 방법에 대...,29303
35228,35246,없다 미래 유망 직종 전기공학,153104
35229,35247,기존 노후화된 기계로 교체하기 위해서 새로운 기계 장비를 도입했음 4차산업혁명으로 ...,154103


In [ ]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [ ]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [ ]:
years = ['2017', '2018', '2019', '2020']

rf_predicts = []
year_encoder = {}

for year, train, test in zip(years, know_train, know_test):
    print(year)
    encoders = {}
    for col in train.columns:
        if col == 'ID':
            continue
        try:
            train[col] = train[col].map(int)
        except:
            encoder = LabelEncoder()
            train[col] = train[col].map(str)
            train[col] = encoder.fit_transform(train[col])
            encoders[col] = encoder  
    year_encoder[year] = encoders

    train = pd.merge(train, train_know[['idx', 'label_answer']], how='left', left_on='idx', right_on='idx')
    train = train[train['label_answer'].notnull()]

    feature = train.columns.drop(['idx', 'knowcode'])

    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(train[feature], train['knowcode'])

    for col in test.columns:
      try:
          test[col] = test[col].map(int)
      except:
          encoder = year_encoder[year][col]
          test[col] = test[col].map(str)
          category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
          test[col] = test[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리
    test = pd.merge(test, test_know[['idx', 'label_answer']], how='left', left_on='idx', right_on='idx')
    feature = test.columns.drop(['idx'])
    pred = model.predict(test[feature])
    rf_predicts.extend(pred)

2017
2018
2019
2020


In [ ]:
submission = pd.read_csv('/content/gdrive/MyDrive/know/sample_submission.csv') # sample submission 불러오기

In [ ]:
submission['knowcode'] = rf_predicts

In [ ]:
submission

,idx,knowcode
0,0,29401
1,1,140306
2,2,122102
3,3,121102
4,4,412003
...,...,...
35226,35244,833001
35227,35245,29902
35228,35246,833001
35229,35247,154101


In [ ]:
model.feature_importances_

array([0.00502974, 0.00696356, 0.0046186 , 0.00618455, 0.00536942,
       0.00654818, 0.00524004, 0.00641235, 0.00490152, 0.00649135,
       0.00469128, 0.00622218, 0.00544991, 0.00656817, 0.00487168,
       0.00627954, 0.00480141, 0.00620925, 0.00538982, 0.00605254,
       0.00460519, 0.00547864, 0.00499422, 0.00571897, 0.0048845 ,
       0.00616185, 0.00464613, 0.00534539, 0.00528793, 0.00615223,
       0.00496407, 0.00592526, 0.00504545, 0.00560038, 0.00469068,
       0.00559563, 0.00485934, 0.00596912, 0.00560499, 0.00667132,
       0.00508969, 0.00585811, 0.00483682, 0.00596638, 0.00490135,
       0.00612964, 0.00481428, 0.00559361, 0.00460386, 0.00543104,
       0.00478782, 0.00520836, 0.00479766, 0.00558719, 0.00495684,
       0.00563603, 0.00495421, 0.006079  , 0.0044449 , 0.00537286,
       0.00538743, 0.00656375, 0.00489197, 0.00552374, 0.00484655,
       0.00573845, 0.00457156, 0.00553981, 0.00463637, 0.00575467,
       0.00484491, 0.00567674, 0.00454581, 0.00534238, 0.00556

In [ ]:
submission.to_csv('submission.csv', index=False)